In [2]:
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [17]:
references_links = pd.read_csv(r'../src/data/raw/references/references_links.csv',
                               sep=';')
extracted_text = []

In [46]:
class GetData:
    def __init__(self):
        pass

    def from_reddit(self, url: str, qtd_chamadas: int=5) -> list[dict]:
        json_result_list = []
        tentativa = qtd_chamadas
        aux = 0

        # Primeira requisição
        response = requests.get(url)
        json_object = response.json()

        next_page = json_object['data']['after']
        data_exists = json_object['data']

        for item in json_object['data']['children']:
            aux = aux+1
        for page in range(aux):
            json_result_list.append(json_object['data']['children'][page]['data']['selftext'])

        # Loop para paginação
        while data_exists:
            if tentativa > 0:
                time.sleep(20)
                print("Next page:", next_page)
                response = requests.get(url, params={'after': next_page})
                json_object = response.json()

                # Pega a chave selftext de cada post, por página
                aux = 0
                for item in json_object['data']['children']:
                    aux = aux+1
                for page in range(aux):
                    json_result_list.append(json_object['data']['children'][page]['data']['selftext'])

                next_page = json_object['data'].get('after')
                data_exists = json_object['data']
                tentativa = tentativa - 1
            else: break
        return json_result_list
          
def from_website(self, references_links: pd.DataFrame) -> pd.DataFrame:
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('--log-level=3')
    extracted_text = []

    driver = webdriver.Chrome(options=options)

    for _, row in references_links.iterrows():
        journal = row['journal']
        link = row['link']
        try:
            driver.get(link)
            time.sleep(5)

            if journal == 'einvestidor':
                xpath = '/html/body/div[2]/article/div[3]'
            elif journal == 'cnnbrasil':
                xpath = '/html/body/div[2]/article/div[3]'
            elif journal == 'valorinveste':
                xpath = '/html/body/div[1]/main/div[2]/div/article/div[1]/div[3]'
            elif journal == 'agenciabrasil':
                xpath = '/html/body/main/div[2]/div/div/div[1]/div/div[5]'
            elif journal == 'estadao':
                xpath = '/html/body/div[1]/main/article'
            elif journal == 'f5folha':
                xpath = '/html/body/div[9]/div/div[1]/article/div[3]'
            elif journal == 'poder360':
                xpath = '/html/body/div[1]/main/div[2]/div/div[1]/article/div[1]/div[3]'
            elif journal == 'g1':
                xpath = '/html/body/div[2]/main/div[4]/article'
            elif journal == 'serasa':
                xpath = '/html/body/div[1]/div/div/main/div/div/div/div/div/div/div[2]/div[1]/div/div[2]/div/div[1]/div[5]'
            elif journal == 'noticiasuol':
                xpath = '/html/body/div[1]/main/article/div[1]/div[2]/div/div/div'
            elif journal == 'em':
                xpath = '/html/body/div[2]/div[5]/div[1]/div[1]/div/div/div/div[4]/div[1]/div/div[4]/div[2]'
            elif journal == 'unicesumar':
                xpath = '/html/body/div[1]/div/div[1]'
            elif journal == 'contraponto':
                xpath = '/html/body/div[1]/div/div/div[2]/div/div[1]/article/div[2]'
            elif journal == 'cm7brasil':
                xpath = '/html/body/main/div/div[2]/div/article'
            elif journal == 'folhafinanceira':
                xpath = '/html/body/div[2]/div/main/div/div/div[1]/div[1]/article/div[4]'
            elif journal == 'jornalcontabil':
                xpath = '/html/body/div[4]/div[2]/div/div/article/div/div/div[1]/div/div[1]/div/div[2]/div/div[3]/div/div[1]/div[1]'
            else:
                print(f"Jornal não mapeado: {journal}")
                continue

            text_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            extracted_text.append(text_element.text)

        except Exception as e:
            print(f"Erro ao carregar texto do site {journal}: {e}")

    driver.quit()
    return extracted_text



In [49]:
do_requests = GetData()

In [48]:
list_data_news = do_requests.from_website(references_links=references_links)
df_list_data_news = pd.DataFrame(list_data_news, columns=['registros_websites'])

AttributeError: 'GetData' object has no attribute 'from_website'

In [39]:

df_list_data_news.to_csv(r'../src/data/processed/df_list_data_news.csv',
                           sep=';')

In [ ]:
list_data_reddit = do_requests.from_reddit(url='https://www.reddit.com/r/golpe.json',
                                    qtd_chamadas=10)
df_list_data_reddit = pd.DataFrame(list_data_reddit, columns=['registros_reddit'])

Next page: t3_1l4gr4p
Next page: t3_1l2v9ob
Next page: t3_1l1uate
Next page: t3_1l0395g
Next page: t3_1kzlczg


KeyError: 'data'

In [36]:
df_list_data_reddit.to_csv(r'../src/data/processed/df_list_data_reddit.csv',
                           sep=';')

In [41]:
df_list_data_reddit

,registros_reddit
0,O sub sempre teve como ideia ajudar e espalhar...
1,Postei uma progressão salarial e ja veio conta...
2,Postando novamente pq não sabia que não podia ...
3,mandei mensagem em uma pizzaria (whatsapp no s...
4,"Post anterior apagado, porque esqueci de borra..."
...,...
521,"Acho muito estranho o cara nem pedir desconto,..."
522,Meu namorado acabou tomando um golpe tentando ...
523,Final de semana uma amiga foi comprar um produ...
524,\n\nEstava com minha namorada no sítio. Lugar ...
